![img](images/pacman.gif)

## Overview

`dlt` is an open-source library that you can add to your Python scripts to load data from various and often messy data sources into well structured, live datasets.

How it works?

`dlt` extracts data from a source, inspects its structure to generate a schema, organises, normalizes and verifies the data, and loads the data into a destination, such as a database.


![img](images/dlt-high-level.png)

Below we give you a preview how you can get data from APIs, files, Python objects or pandas dataframes and move it into a local or remote database, data lake or a vector data store. 

Let's get started!

## Installation

Official releases of dlt can be installed from [PyPI](https://pypi.org/project/dlt/):

In [8]:
!pip install -q dlt

Command above just installs library core, in example below we use `duckdb` as a [destination](https://dlthub.com/docs/dlt-ecosystem/destinations), so let's add it:

In [7]:
!pip install -q "dlt[duckdb]"

> Use clean virtual environment for your experiments! Here are [detailed instructions](https://dlthub.com/docs/reference/installation).

## Quick start

Let's load a list of Python objects (dicts) into `duckdb` database and inspect the created dataset.

> We gonna use `full_refresh` for our test examples. If you create a new pipeline script you will be experimenting a lot. If you want that each time the pipeline resets its state and loads data to a new dataset, set the full_refresh argument of the dlt.pipeline method to True. Each time the pipeline is created, dlt adds datetime-based suffix to the dataset name.

In [1]:
import dlt

data = [
	{'id': 1, 'name': 'Alice'},
	{'id': 2, 'name': 'Bob'}
]

pipeline = dlt.pipeline(
	pipeline_name='quick_start',
	destination='duckdb',
	dataset_name='mydata',
    full_refresh=True, 
)
load_info = pipeline.run(data, table_name="users")
print(load_info)

Pipeline quick_start completed in 1.26 seconds
1 load package(s) were loaded to destination duckdb and into dataset mydata_20230902032147
The duckdb destination used duckdb:////Users/alenaastrakhantseva/PycharmProjects/dlthub/spotlight_demo/quick_start.duckdb location to store data
Load package 1693668107.411324 is LOADED and contains no failed jobs


### Now explore your data! 

To see the schema of your created database, run Streamlit command:

```python
 dlt pipeline <pipeline_name> show
```
[This command](https://dlthub.com/docs/reference/command-line-interface#show-tables-and-data-in-the-destination) generates and launches a simple Streamlit app that you can use to inspect the schemas and data in the destination.

To use `streamlit`, install it first.

For example above pipeline name is “quick_start”, so run:

In [2]:
!pip install -q streamlit

In [ ]:
!dlt pipeline quick_start show

/Users/alenaastrakhantseva/venvs/spotlight_demo/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Found pipeline quick_start in /Users/alenaastrakhantseva/.dlt/pipelines

  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://192.168.178.73:8501

  For better performance, install the Watchdog module:

  $ xcode-select --install
  $ pip install watchdog
            


## Load data from variety of sources

Use dlt to load practically any data you deal with in your Python script into a dataset. 

The library will create/update tables, infer data types and deal with nested data automatically:
- list of dicts
- json
- csv
- API
- database
- etc.

### from JSON

When creating a schema during normalization, dlt recursively unpacks this nested structure into relational tables, creating and linking [children and parent tables](https://dlthub.com/docs/dlt-ecosystem/visualizations/understanding-the-tables#child-and-parent-tables).

In [37]:
# create test json file

import json

with open("test.json", 'w') as file:
    data = {
        'id': 1, 
        'name': 'Alice', 
        'children': {
            'id': 1, 
            'name': 'Eve'
        }
    }
    json.dump(data, file)


In [62]:
# load test json to duckdb database

import json
import dlt


with open("test.json", 'r') as file:
    data = json.load(file)


pipeline = dlt.pipeline(
	pipeline_name='from_json',
	destination='duckdb', 
	dataset_name='mydata',
    full_refresh=True,
)
# dlt works with lists of dicts, so wrap data to the list
load_info = pipeline.run([data], table_name="json_data")
print(load_info)

Pipeline from_json completed in 0.59 seconds
1 load package(s) were loaded to destination duckdb and into dataset mydata_20230828060445
The duckdb destination used duckdb:////Users/alenaastrakhantseva/notebooks/dlthub/dlt_demo/from_json.duckdb location to store data
Load package 1693245886.056222 is LOADED and contains no failed jobs


In [63]:
import duckdb

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
display(conn.sql("DESCRIBE"))
data_table = conn.sql("SELECT * FROM json_data").df()
data_table

┌───────────┬──────────────────────┬─────────────────────┬──────────────────────┬──────────────────────────┬───────────┐
│ database  │        schema        │        name         │     column_names     │       column_types       │ temporary │
│  varchar  │       varchar        │       varchar       │      varchar[]       │        varchar[]         │  boolean  │
├───────────┼──────────────────────┼─────────────────────┼──────────────────────┼──────────────────────────┼───────────┤
│ from_json │ mydata_20230828043…  │ _dlt_loads          │ [load_id, schema_n…  │ [VARCHAR, VARCHAR, BIG…  │ false     │
│ from_json │ mydata_20230828043…  │ _dlt_pipeline_state │ [version, engine_v…  │ [BIGINT, BIGINT, VARCH…  │ false     │
│ from_json │ mydata_20230828043…  │ _dlt_version        │ [version, engine_v…  │ [BIGINT, BIGINT, TIMES…  │ false     │
│ from_json │ mydata_20230828043…  │ json_data           │ [id, name, childre…  │ [BIGINT, VARCHAR, BIGI…  │ false     │
│ from_json │ mydata_20230828060

,id,name,children__id,children__name,_dlt_load_id,_dlt_id
0,1,Alice,1,Eve,1693245886.056222,i1/pvQQcJBvobw


### from API

Below we load 100 most recent followers from our [own dlt-hub organisation](https://github.com/dlt-hub/dlt) into "followers" table.

In [76]:
import dlt
import requests


# url to request dlt-hub followers
url = f"https://api.github.com/users/dlt-hub/followers"
# make the request and return the json
data = requests.get(url).json()

pipeline = dlt.pipeline(
	pipeline_name='from_api',
	destination='duckdb', 
	dataset_name='mydata',
    full_refresh=True,
)
load_info = pipeline.run(data, table_name="followers")
print(load_info)

Pipeline from_api completed in 0.57 seconds
1 load package(s) were loaded to destination duckdb and into dataset mydata_20230828060613
The duckdb destination used duckdb:////Users/alenaastrakhantseva/notebooks/dlthub/dlt_demo/from_api.duckdb location to store data
Load package 1693245973.421062 is LOADED and contains no failed jobs


In [77]:
import duckdb

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
data_table = conn.sql("SELECT * FROM followers").df()
data_table.head()

,login,id,node_id,avatar_url,gravatar_id,url,html_url,followers_url,following_url,gists_url,starred_url,subscriptions_url,organizations_url,repos_url,events_url,received_events_url,type,site_admin,_dlt_load_id,_dlt_id
0,esneko,64329,MDQ6VXNlcjY0MzI5,https://avatars.githubusercontent.com/u/64329?v=4,,https://api.github.com/users/esneko,https://github.com/esneko,https://api.github.com/users/esneko/followers,https://api.github.com/users/esneko/following{...,https://api.github.com/users/esneko/gists{/gis...,https://api.github.com/users/esneko/starred{/o...,https://api.github.com/users/esneko/subscriptions,https://api.github.com/users/esneko/orgs,https://api.github.com/users/esneko/repos,https://api.github.com/users/esneko/events{/pr...,https://api.github.com/users/esneko/received_e...,User,False,1693245973.421062,UqVsp5Dy/LFQjg
1,rossmason,148282,MDQ6VXNlcjE0ODI4Mg==,https://avatars.githubusercontent.com/u/148282...,,https://api.github.com/users/rossmason,https://github.com/rossmason,https://api.github.com/users/rossmason/followers,https://api.github.com/users/rossmason/followi...,https://api.github.com/users/rossmason/gists{/...,https://api.github.com/users/rossmason/starred...,https://api.github.com/users/rossmason/subscri...,https://api.github.com/users/rossmason/orgs,https://api.github.com/users/rossmason/repos,https://api.github.com/users/rossmason/events{...,https://api.github.com/users/rossmason/receive...,User,False,1693245973.421062,imX+sB7ectnkhw
2,csakshaug,200077,MDQ6VXNlcjIwMDA3Nw==,https://avatars.githubusercontent.com/u/200077...,,https://api.github.com/users/csakshaug,https://github.com/csakshaug,https://api.github.com/users/csakshaug/followers,https://api.github.com/users/csakshaug/followi...,https://api.github.com/users/csakshaug/gists{/...,https://api.github.com/users/csakshaug/starred...,https://api.github.com/users/csakshaug/subscri...,https://api.github.com/users/csakshaug/orgs,https://api.github.com/users/csakshaug/repos,https://api.github.com/users/csakshaug/events{...,https://api.github.com/users/csakshaug/receive...,User,False,1693245973.421062,5dcfjDECQoqtlA
3,hvignolo87,205441,MDQ6VXNlcjIwNTQ0MQ==,https://avatars.githubusercontent.com/u/205441...,,https://api.github.com/users/hvignolo87,https://github.com/hvignolo87,https://api.github.com/users/hvignolo87/followers,https://api.github.com/users/hvignolo87/follow...,https://api.github.com/users/hvignolo87/gists{...,https://api.github.com/users/hvignolo87/starre...,https://api.github.com/users/hvignolo87/subscr...,https://api.github.com/users/hvignolo87/orgs,https://api.github.com/users/hvignolo87/repos,https://api.github.com/users/hvignolo87/events...,https://api.github.com/users/hvignolo87/receiv...,User,False,1693245973.421062,qHV+8/VYK0W5uA
4,radik,440621,MDQ6VXNlcjQ0MDYyMQ==,https://avatars.githubusercontent.com/u/440621...,,https://api.github.com/users/radik,https://github.com/radik,https://api.github.com/users/radik/followers,https://api.github.com/users/radik/following{/...,https://api.github.com/users/radik/gists{/gist...,https://api.github.com/users/radik/starred{/ow...,https://api.github.com/users/radik/subscriptions,https://api.github.com/users/radik/orgs,https://api.github.com/users/radik/repos,https://api.github.com/users/radik/events{/pri...,https://api.github.com/users/radik/received_ev...,User,False,1693245973.421062,scP1x7f73Q+9+A


## Append or replace your data

Run this examples twice and you notice that each time a copy of the data is added to your tables.
We call this load mode `append`. It is very useful when i.e. you have a new folder created daily with `json` file logs, and you want to ingest them.

In [68]:
import dlt


data = [
	{'id': 1, 'name': 'Alice'},
	{'id': 2, 'name': 'Bob'}
]

pipeline = dlt.pipeline(
	pipeline_name='quick_start',
	destination='duckdb',
	dataset_name='mydata',
    full_refresh=False, 
)
load_info = pipeline.run(data, table_name="users")
print(load_info)

Pipeline quick_start completed in 0.41 seconds
1 load package(s) were loaded to destination duckdb and into dataset mydata
The duckdb destination used duckdb:////Users/alenaastrakhantseva/notebooks/dlthub/dlt_demo/quick_start.duckdb location to store data
Load package 1693245933.818205 is LOADED and contains no failed jobs


In [69]:
import duckdb

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
data_table = conn.sql("SELECT * FROM users").df()
data_table

,id,name,_dlt_load_id,_dlt_id
0,1,Alice,1693245858.057515,ZNbIiP3i/ELX2Q
1,2,Bob,1693245858.057515,YT9UbcBg4UlIVg
2,1,Alice,1693245920.257415,Q02LBxwC6oW9/g
3,2,Bob,1693245920.257415,UclCCMLraCdCKw
4,1,Alice,1693245933.818205,LyMXT2t5EiDuUA
5,2,Bob,1693245933.818205,yDutzoZOz45DSA


Perhaps this is not what you want to do in the example above.
For example, if the CSV file is updated, how we can refresh it in the database?
One method is to tell `dlt` to replace the data in existing tables by using `write_disposition`.

In [70]:
import dlt


data = [
	{'id': 1, 'name': 'Alice'},
	{'id': 2, 'name': 'Bob'}
]

pipeline = dlt.pipeline(
	pipeline_name='quick_start',
	destination='duckdb',
	dataset_name='mydata',
    full_refresh=False, 
)
load_info = pipeline.run(data, table_name="users", write_disposition="replace")
print(load_info)

Pipeline quick_start completed in 0.37 seconds
1 load package(s) were loaded to destination duckdb and into dataset mydata
The duckdb destination used duckdb:////Users/alenaastrakhantseva/notebooks/dlthub/dlt_demo/quick_start.duckdb location to store data
Load package 1693245938.264698 is LOADED and contains no failed jobs


In [71]:
import duckdb

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
data_table = conn.sql("SELECT * FROM users").df()
data_table

,id,name,_dlt_load_id,_dlt_id
0,1,Alice,1693245938.264698,kD167b87zS+PWA
1,2,Bob,1693245938.264698,bK5tWWHVj0lF2A


## Declare loading behavior

You can finetune the loading process by decorating Python functions with `@dlt.resource`.

### Load only new data (incremental loading)

We can supercharge the GitHub API example above and get only issues that were created since last load.
Instead of using `replace` write_disposition and downloading all issues each time the pipeline is run, we do the following:

In [72]:
import dlt
from dlt.sources.helpers import requests


@dlt.resource(table_name="issues", write_disposition="append")
def get_issues(
    created_at=dlt.sources.incremental("created_at", initial_value="1970-01-01T00:00:00Z")
):
    # url to request dlt-hub issues
    url = f"https://api.github.com/repos/dlt-hub/dlt/issues"

    while True:
        response = requests.get(url)
        page_items = response.json()

        if len(page_items) == 0:
            break
        yield page_items

        if "next" not in response.links:
            break
        url = response.links["next"]["url"]

        # stop requesting pages if the last element was already older than initial value
        # note: incremental will skip those items anyway, we just do not want to use the api limits
        if created_at.start_out_of_range:
            break


pipeline = dlt.pipeline(
    pipeline_name='github_issues',
    destination='duckdb',
    dataset_name='mydata',
)
# dlt works with lists of dicts, so wrap data to the list
load_info = pipeline.run(get_issues)
print(load_info)

Pipeline github_issues completed in 2.39 seconds
1 load package(s) were loaded to destination duckdb and into dataset mydata
The duckdb destination used duckdb:////Users/alenaastrakhantseva/notebooks/dlthub/dlt_demo/github_issues.duckdb location to store data
Load package 1693245946.643318 is LOADED and contains no failed jobs


We request issues for dlt-hub/dlt repository ordered by **created_at** descending and yield them page by page in `get_issues` generator function.

We use the `@dlt.resource` decorator to declare table name to which data will be loaded and write disposition, which is `append`.

We also use `dlt.sources.incremental` to track `created_at` field present in each issue to filter only the newly created ones.

Now run the script. It loads all the issues from our repo to `duckdb`. Run it again, and you can see that no issues got added (if no issues were created in the meantime).

Now you can run this script on a daily schedule and each day you’ll load only issues created after the time of the previous pipeline run.

In [74]:
import duckdb

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
data_table = conn.sql("SELECT * FROM issues").df()
data_table.head()

,url,repository_url,labels_url,comments_url,events_url,html_url,id,node_id,number,title,...,assignee__gists_url,assignee__starred_url,assignee__subscriptions_url,assignee__organizations_url,assignee__repos_url,assignee__events_url,assignee__received_events_url,assignee__type,assignee__site_admin,body
0,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt/issue...,https://github.com/dlt-hub/dlt/pull/594,1868543406,PR_kwDOGvRYu85Y3uiK,594,[wip] basic schema freezing,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt/issue...,https://github.com/dlt-hub/dlt/pull/592,1868279717,PR_kwDOGvRYu85Y28S1,592,Add azure blob storage filesystem/staging dest...,...,https://api.github.com/users/sh-rp/gists{/gist...,https://api.github.com/users/sh-rp/starred{/ow...,https://api.github.com/users/sh-rp/subscriptions,https://api.github.com/users/sh-rp/orgs,https://api.github.com/users/sh-rp/repos,https://api.github.com/users/sh-rp/events{/pri...,https://api.github.com/users/sh-rp/received_ev...,User,False,Resolves https://github.com/dlt-hub/dlt/issues...
2,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt/issue...,https://github.com/dlt-hub/dlt/pull/590,1867720041,PR_kwDOGvRYu85Y1Lnv,590,uses botocore instead of boto3 in AwsCredentials,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt/issue...,https://github.com/dlt-hub/dlt/pull/589,1867504604,PR_kwDOGvRYu85Y0cyW,589,Add pydantic support,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://github.com/dlt-hub/dlt/issues/501
4,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt/issue...,https://github.com/dlt-hub/dlt/issues/588,1867354840,I_kwDOGvRYu85vTZLY,588,Full refresh mode adjustment,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Multiple users get confused by full refresh mo...


## Update and deduplicate your data

The script above finds new issues and adds them to the database.
It will ignore any updates to issue text, emoji reactions etc.
Get always fresh content of all the issues: combine incremental load with `merge` write disposition,
like in the script below.

In [83]:
import dlt
import requests


@dlt.resource(
    table_name="issues",
    write_disposition="merge",
    primary_key="id",
)
def get_issues(
    updated_at = dlt.sources.incremental("updated_at", initial_value="1970-01-01T00:00:00Z")
):
    # url to request dlt-hub issues
    url = f"https://api.github.com/repos/dlt-hub/dlt/issues?since={updated_at.last_value}"

    while True:
        response = requests.get(url)
        page_items = response.json()

        if len(page_items) == 0:
            break
        yield page_items

        if "next" not in response.links:
            break
        url = response.links["next"]["url"]


pipeline = dlt.pipeline(
    pipeline_name='github_issues_merge',
    destination='duckdb',
    dataset_name='mydata',
    full_refresh=True,
)
# dlt works with lists of dicts, so wrap data to the list
load_info = pipeline.run(get_issues)
print(load_info)

Pipeline github_issues_merge completed in 3.28 seconds
1 load package(s) were loaded to destination duckdb and into dataset mydata_20230828062204
The duckdb destination used duckdb:////Users/alenaastrakhantseva/notebooks/dlthub/dlt_demo/github_issues_merge.duckdb location to store data
Load package 1693246926.880802 is LOADED and contains no failed jobs


Above we add `primary_key` hint that tells `dlt` how to identify the issues in the database to find duplicates which content it will merge.

Note that we now track the `updated_at` field - so we filter in all issues **updated** since the last pipeline run (which also includes newly created ones).

Also pay attention how we use **since** [GitHub API](https://docs.github.com/en/rest/issues/issues?apiVersion=2022-11-28#list-repository-issues)
and `updated_at.last_value` to tell GitHub which issues we are interested in. `updated_at.last_value` holds the last `updated_at` value from the previous run.

In [82]:
import duckdb

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
display(conn.sql("DESCRIBE"))
data_table = conn.sql("SELECT * FROM issues").df()
data_table.head()

┌─────────────────────┬──────────────────────┬─────────────────────┬───┬──────────────────────┬───────────┐
│      database       │        schema        │        name         │ … │     column_types     │ temporary │
│       varchar       │       varchar        │       varchar       │   │      varchar[]       │  boolean  │
├─────────────────────┼──────────────────────┼─────────────────────┼───┼──────────────────────┼───────────┤
│ github_issues_merge │ mydata_20230828062…  │ _dlt_loads          │ … │ [VARCHAR, VARCHAR,…  │ false     │
│ github_issues_merge │ mydata_20230828062…  │ _dlt_pipeline_state │ … │ [BIGINT, BIGINT, V…  │ false     │
│ github_issues_merge │ mydata_20230828062…  │ _dlt_version        │ … │ [BIGINT, BIGINT, T…  │ false     │
│ github_issues_merge │ mydata_20230828062…  │ issues              │ … │ [BIGINT, VARCHAR, …  │ false     │
│ github_issues_merge │ mydata_20230828062…  │ issues__assignees   │ … │ [VARCHAR, BIGINT, …  │ false     │
│ github_issues_merge │ myda

,id,url,repository_url,labels_url,comments_url,events_url,html_url,node_id,number,title,...,assignee__gists_url,assignee__starred_url,assignee__subscriptions_url,assignee__organizations_url,assignee__repos_url,assignee__events_url,assignee__received_events_url,assignee__type,assignee__site_admin,body
0,1868543406,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt/issue...,https://github.com/dlt-hub/dlt/pull/594,PR_kwDOGvRYu85Y3uiK,594,[wip] basic schema freezing,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1868279717,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt/issue...,https://github.com/dlt-hub/dlt/pull/592,PR_kwDOGvRYu85Y28S1,592,Add azure blob storage filesystem/staging dest...,...,https://api.github.com/users/sh-rp/gists{/gist...,https://api.github.com/users/sh-rp/starred{/ow...,https://api.github.com/users/sh-rp/subscriptions,https://api.github.com/users/sh-rp/orgs,https://api.github.com/users/sh-rp/repos,https://api.github.com/users/sh-rp/events{/pri...,https://api.github.com/users/sh-rp/received_ev...,User,False,Resolves https://github.com/dlt-hub/dlt/issues...
2,1867720041,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt/issue...,https://github.com/dlt-hub/dlt/pull/590,PR_kwDOGvRYu85Y1Lnv,590,uses botocore instead of boto3 in AwsCredentials,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1867354840,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt/issue...,https://github.com/dlt-hub/dlt/issues/588,I_kwDOGvRYu85vTZLY,588,Full refresh mode adjustment,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Multiple users get confused by full refresh mo...
4,1867045017,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt/issue...,https://github.com/dlt-hub/dlt/issues/586,I_kwDOGvRYu85vSNiZ,586,Feature request: dbt cloud runner,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,We want to equally support dbt core and cloud....


### Use existed verified sources

To use existed verified source, just run the `dlt init` [command](https://dlthub.com/docs/reference/command-line-interface#dlt-init).

List all verified sources:

In [89]:
!dlt init --list-verified-sources

Looking up for verified sources in https://github.com/dlt-hub/verified-sources.git...
mux: Loads Mux views data using https://docs.mux.com/api-reference
google_sheets: Loads Google Sheets data from tabs, named and explicit ranges. Contains the main source functions.
google_analytics: Defines all the sources and resources needed for Google Analytics V4
pokemon: This source provides data extraction from an example source as a starting point for new pipelines.
pipedrive: Highly customizable source for Pipedrive, supports endpoint addition, selection and column rename
workable: This source uses Workable API and dlt to load data such as Candidates, Jobs, Events, etc. to the database.
sql_database: Source that loads tables form any SQLAlchemy supported database, supports batching requests and incremental loads.
facebook_ads: Loads campaigns, ads sets, ads, leads and insight data from Facebook Marketing API
notion: A source that extracts data from Notion API
hubspot: This is a module that pro

This command shows all available verified sources and their short descriptions. For each source, checks if your local `dlt` version requires update and prints the relevant warning.

Consider an example of a pipeline for an pokemon.

This command will initialize the pipeline example with Pokemon as the source and `duckdb` as the [destination](https://dlthub.com/docs/dlt-ecosystem/destinations):


In [90]:
!dlt --non-interactive init pokemon duckdb

Looking up the init scripts in https://github.com/dlt-hub/verified-sources.git...
Cloning and configuring a verified source pokemon (This source provides data extraction from an example source as a starting point for new pipelines.)

Verified source pokemon was added to your project!
* See the usage examples and code snippets to copy from pokemon_pipeline.py
* Add credentials for duckdb and other secrets in ./.dlt/secrets.toml
* requirements.txt was created. Install it with:
pip3 install -r requirements.txt
* Read https://dlthub.com/docs/walkthroughs/create-a-pipeline for more information


In [70]:
!python pokemon_pipeline.py

Pipeline pokemon completed in 1.57 seconds
1 load package(s) were loaded to destination duckdb and into dataset pokemon_data
The duckdb destination used duckdb:////Users/alenaastrakhantseva/notebooks/dlthub/dlt_demo/pokemon.duckdb location to store data
Load package 1692810489.812506 is LOADED and contains no failed jobs


In [ ]:
!dlt pipeline pokemon show

Found pipeline pokemon in /Users/alenaastrakhantseva/.dlt/pipelines

  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://192.168.2.219:8501

  For better performance, install the Watchdog module:

  $ xcode-select --install
  $ pip install watchdog
            


In [ ]:
import duckdb

conn = duckdb.connect(f"pokemon.duckdb")
conn.sql(f"SET search_path = 'pokemon_data'")
display(conn.sql("DESCRIBE"))
data_table = conn.sql("SELECT * FROM pokemon").df()
data_table